# Import

In [1]:
import pandas as pd 
from urllib.request import urlopen
from dotenv import dotenv_values
from sqlalchemy import create_engine, types
from sqlalchemy.dialects.postgresql import JSON as postgres_json
import pandas as pd
import requests
import json
from datetime import datetime
import time
import pprint

# Reading Session Keys

In [2]:
session_key_list = pd.read_csv('session_keys.csv')

In [3]:
for i in session_key_list['session_key']:
    response = urlopen(f'https://api.openf1.org/v1/position?session_key={i}')
    data = json.loads(response.read().decode('utf-8'))
    if i == 7953:
        df_position = pd.DataFrame(data)
    else:
        hold = pd.DataFrame(data)
        df_position = pd.concat([df_position,hold])
        time.sleep(0.5)
    print(f'Session {i} extracted.')
print('Done')

Session 7953 extracted.
Session 7779 extracted.
Session 7787 extracted.
Session 9070 extracted.
Session 9078 extracted.
Session 9094 extracted.
Session 9102 extracted.
Session 9110 extracted.
Session 9118 extracted.
Session 9126 extracted.
Session 9133 extracted.
Session 9141 extracted.
Session 9149 extracted.
Session 9157 extracted.
Session 9165 extracted.
Session 9173 extracted.
Session 9221 extracted.
Session 9213 extracted.
Session 9181 extracted.
Session 9205 extracted.
Session 9189 extracted.
Session 9197 extracted.
Session 9472 extracted.
Session 9480 extracted.
Session 9488 extracted.
Session 9496 extracted.
Session 9673 extracted.
Session 9507 extracted.
Session 9515 extracted.
Session 9523 extracted.
Session 9531 extracted.
Session 9539 extracted.
Session 9550 extracted.
Session 9558 extracted.
Session 9566 extracted.
Session 9574 extracted.
Session 9582 extracted.
Session 9590 extracted.
Session 9598 extracted.
Session 9606 extracted.
Session 9617 extracted.
Session 9625 ext

In [4]:
df_position

,date,session_key,meeting_key,driver_number,position
0,2023-03-05T14:01:06.787000+00:00,7953,1141,1,1
1,2023-03-05T14:01:06.787000+00:00,7953,1141,11,2
2,2023-03-05T14:01:06.787000+00:00,7953,1141,16,3
3,2023-03-05T14:01:06.787000+00:00,7953,1141,55,4
4,2023-03-05T14:01:06.787000+00:00,7953,1141,14,5
...,...,...,...,...,...
324,2025-11-23T05:09:27.920000+00:00,9858,1274,43,17
325,2025-11-23T05:18:36.798000+00:00,9858,1274,10,15
326,2025-11-23T05:18:36.798000+00:00,9858,1274,22,14
327,2025-11-23T05:25:44.786000+00:00,9858,1274,81,4


In [5]:
config = dotenv_values()

pg_user = config['POSTGRES_USER']
pg_pass = config['POSTGRES_PASS']
pg_host = config['POSTGRES_HOST']
pg_port = config['POSTGRES_PORT']
pg_db = config['POSTGRES_DB']
pg_schema = config['POSTGRES_SCHEMA']

url = f'postgresql://{pg_user}:{pg_pass}@{pg_host}:{pg_port}/{pg_db}'
engine = create_engine(url, echo=False)

In [6]:
df_position.columns

Index(['date', 'session_key', 'meeting_key', 'driver_number', 'position'], dtype='object')

In [7]:
dtype_dict = {
    'date': types.DateTime,
    'session_key': types.Integer,
    'meeting_key': types.Integer,
    'driver_number': types.Integer,
    'position': types.Integer
}

In [8]:
df_position.to_sql(name = 'raw_position', 
                       con = engine, 
                       schema = pg_schema, 
                       if_exists='replace', 
                       dtype=dtype_dict,
                       index=False
                      )

940